In [7]:
import requests
from bs4 import BeautifulSoup
import re

def fetch_page(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, 'html.parser')

def extract_details(soup):
    details = {}
    tables = soup.find_all("table", class_="table table-striped table-hover")
    if tables:
        general_table = tables[0]
        for row in general_table.find_all("tr"):
            cells = row.find_all("td")
            if cells:
                details[cells[0].text.strip()] = cells[1].text.strip()

        if len(tables) > 1:
            memory_table = tables[1]
            for row in memory_table.find_all("tr"):
                cells = row.find_all("td")
                if cells:
                    details[cells[0].text.strip()] = cells[1].text.strip()

    return details

def extract_price(soup):
    price_text = soup.find("div", class_="pprice").text.strip()
    price_match = re.search(r"\d+,\d+", price_text)
    if price_match:
        price = price_match.group(0).replace(',', '.')  # Ersetzen von Komma durch Punkt
        return float(price)
    return None

def get_graphics_card_data(url):
    soup = fetch_page(url)
    details = extract_details(soup)
    price = extract_price(soup)
    return {
        "Name": details.get("GPU Modell:"),
        "Taktfrequenz": details.get("GPU Takt:"),
        "Streamprozessoren": details.get("Anzahl der Streamprozessoren:"),
        "Kapazität": details.get("Grösse des Grafikspeichers:"),
        "Anbindung": details.get("Grafikspeicher Anbindung:"),
        "Effektiver Takt": details.get("Grafikspeicher Taktfrequenz:"),
        "Preis": price
    }

def print_graphics_card_data(data):
    print("Grafikkarten-Daten:")
    for key, value in data.items():
        if key == "Preis":
            print(f"{key}: {value} €")
        else:
            print(f"{key}: {value}")

# URL der Grafikkarte
url = "https://www.mindfactory.de/product_info.php/16GB-MSI-GeForce-RTX-4070-Ti-SUPER-Gaming-X-Slim-Aktiv-PCIe-4-0-x16-1xH_1521175.html"
data = get_graphics_card_data(url)
print_graphics_card_data(data)


Grafikkarten-Daten:
Name: GeForce RTX 4070 Ti SUPER
Taktfrequenz: 2340MHz
Streamprozessoren: 8448 Einheiten
Kapazität: 16GB
Anbindung: 256Bit
Effektiver Takt: 1400MHz (22400MHz GDDR6X)
Preis: 898.37 €
